In [1]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from tqdm.notebook import tnrange,tqdm
import pandas as pd
import os

In [2]:
from diffusion_simulation import diffusion_simulation
from network import network

## Generate synthetic network instances 

### __Example 1__: Generate one ER graph with size 100, density 0.1

In [34]:
n = 100 # Network size, choose from [20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
pn = '1' # Network density, choose from ['1_n13', '1_n11', '1_n', 'sqrtlogn_n', 'logn_n', 'logn2_n', '1']
rep = 1 # For illustration, generate one repetition. To fully replicate the result in the paper, change this to 50.

for i in range(1,rep+1):
    sim_id = f'ER_{pn}/n{n}_{i}'
    os.makedirs(f'../instances/{sim_id}', exist_ok=True)
    params = {'sim_id':sim_id, 'alpha':0.1, 'beta':3.5,
              'noise_dist': 'logistic', 'noise_dist_param':1,
              'cnt_iter': 100000, 't_warmup':1000, 'is_traj':False, 
              'network_type': 'ER',
              'is_network_given':False, 'network_size': n, 'ER_prob':0.1, 'network_dir':'',
              'is_value_given':False, 'v_dist':'uniform_neg', 'v_dist_param': 4, 'value_dir':''
             }
    
    G = network(params)
    sample = diffusion_simulation(G,params)
    sample.start_diffusion()
    sample.run_fixed_point()
    sample.output()

Instance generated with 100 nodes.


In [36]:
# Figure 3, illustration for one of the replications of a specific (n,p(n)) ER random graph

sim_id = f'ER_{pn}/n{n}_{1}'

df = pd.read_csv(f'../instances/{sim_id}/results.csv')
params = {'sim_id':sim_id, 'alpha':0.1, 'beta':3.5,
          'noise_dist': 'logistic', 'noise_dist_param': 1,
          'cnt_iter': 100000, 't_warmup':1000, 'is_traj':False, 
          'is_network_given':True, 'network_dir':f'../instances/{sim_id}/edge.csv',  
          'is_value_given':True, 'value_dir':f'../instances/{sim_id}/v.csv'
         }
G = network(params)

is_not_0 = G.in_degree > 0
if G.n<len(df):
    df = df[:G.n]
df_1 = df[is_not_0]
mape = np.mean(np.abs(df_1['sim']-df_1['fp'])/df_1['sim'])
    
G.cal_mean_inv_indeg()

print('MAPE:', mape*100) # Figure 3(a)
print('Inverse in-degree density:', np.mean(1/G.in_degree[is_not_0])) # Figure 3(b)
print('Inverse of average in-degree:', 1/np.mean(G.in_degree)) # Figure 3(c)

Instance generated with 100 nodes.
MAPE: 3.6896278293831886
Inverse in-degree density: 0.11699688
Inverse of average in-degree: 0.1027749277540214


### __Example 2__: Generate one ER graph with size 1000, density $\log{n}/n$

In [7]:
n = 1000 # Network size, choose from [20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
pn = 'logn_n' # Network density, choose from ['1_n13', '1_n11', '1_n', 'sqrtlogn_n', 'logn_n', 'logn2_n', '1']
rep = 1 # For illustration, generate one repetition. To fully replicate the result in the paper, change this to 50.

for i in range(1,rep+1):
    sim_id = f'ER_{pn}/n{n}_{i}'
    os.makedirs(f'../instances/{sim_id}', exist_ok=True)
    params = {'sim_id':sim_id, 'alpha':0.1, 'beta':3.5,
              'noise_dist': 'logistic', 'noise_dist_param':1,
              'cnt_iter': 100000, 't_warmup':1000, 'is_traj':False, 
              'network_type': 'ER',
              'is_network_given':False, 'network_size': n, 'ER_prob':np.log(n)/n, 'network_dir':'',
              'is_value_given':False, 'v_dist':'uniform_neg', 'v_dist_param': 4, 'value_dir':''
             }
    
    G = network(params)
    sample = diffusion_simulation(G,params)
    sample.start_diffusion()
    sample.run_fixed_point()
    sample.output()

Instance generated with 1000 nodes.


In [8]:
# Figure 4, illustration for one of the replications of a specific (n,p(n)) ER random graph

sim_id = f'ER_{pn}/n{n}_{1}'

df = pd.read_csv(f'../instances/{sim_id}/results.csv')
params = {'sim_id':sim_id, 'alpha':0.1, 'beta':3.5,
          'noise_dist': 'logistic', 'noise_dist_param': 1,
          'cnt_iter': 100000, 't_warmup':1000, 'is_traj':False, 
          'is_network_given':True, 'network_dir':f'../instances/{sim_id}/edge.csv',  
          'is_value_given':True, 'value_dir':f'../instances/{sim_id}/v.csv'
         }
G = network(params)

is_not_0 = G.in_degree > 0
if G.n<len(df):
    df = df[:G.n]
df_1 = df[is_not_0]
mape = np.mean(np.abs(df_1['sim']-df_1['fp'])/df_1['sim'])

print('MAPE:', mape*100) # Figure 4(a)
print('Inverse in-degree density:', np.mean(1/G.in_degree[is_not_0])) # Figure 4(b)
print('Inverse of average in-degree:', 1/np.mean(G.in_degree)) # Figure 4(c)

Instance generated with 1000 nodes.
MAPE: 6.60778471532297
Inverse in-degree density 0.17600192
Inverse of average in-degree 0.1440299615549603


### __Example 3__: Generate one PL graph with size 100, exponent 2.5, degree correlation 0

In [29]:
n = 100 # Network size, choose from [20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
pn = 'alpha25_theta0' # Network density, choose from ['1_n13', '1_n11', '1_n', 'sqrtlogn_n', 'logn_n', 'logn2_n', '1']
expon, corr, xmin = 3, 0, 2
rep = 1 # For illustration, generate one repetition. To fully replicate the result in the paper, change this to 50.

for i in range(1,rep+1):
    sim_id = f'PL_alpha{expon}_theta{corr}/n{n}_{i}'
    os.makedirs(f'../instances/{sim_id}', exist_ok=True)
    params = {'sim_id':sim_id, 'alpha':0.1, 'beta':3.5,
              'noise_dist': 'logistic', 'noise_dist_param':1,
              'cnt_iter': 100000, 't_warmup':1000, 'is_traj':False, 
              'network_type': 'PL',
              'is_network_given':False, 'network_size': n, 'PL_exponent':expon, 'PL_corr':corr, 'PL_xmin':xmin, 'network_dir':'',
              'is_value_given':False, 'v_dist':'uniform_neg', 'v_dist_param': 4, 'value_dir':''
             }
    
    G = network(params)
    sample = diffusion_simulation(G,params)
    sample.start_diffusion()
    sample.run_fixed_point()
    sample.output()

Instance generated with 100 nodes.


In [31]:
# Figure 5, illustration for one of the replications of a specific (n,p(n)) ER random graph

sim_id = f'PL_alpha{expon}_theta{corr}/n{n}_{1}'

df = pd.read_csv(f'../instances/{sim_id}/results.csv')
params = {'sim_id':sim_id, 'alpha':0.1, 'beta':3.5,
          'noise_dist': 'logistic', 'noise_dist_param': 1,
          'cnt_iter': 100000, 't_warmup':1000, 'is_traj':False, 
          'is_network_given':True, 'network_dir':f'../instances/{sim_id}/edge.csv',  
          'is_value_given':True, 'value_dir':f'../instances/{sim_id}/v.csv'
         }
G = network(params)

mape = np.mean(np.abs(df['sim']-df['fp'])/df['sim'])

print('MAPE:', mape*100) # Figure 5
print('Inverse in-degree density:', np.mean(1/G.in_degree)) # Figure 5

Instance generated with 100 nodes.
MAPE: 14.989297687283692
Inverse in-degree density: 0.40200797


## Generate real-world network instances

In [13]:
sim, folder = 'Caltech36', 'caltech'
os.makedirs(f'../instances/{folder}', exist_ok=True)
df = pd.read_csv(f'../instances/{sim}.txt',sep=' ', names=['from_edge', 'to_edge'])
df_1 = df.copy()
df_1['from_edge'], df_1['to_edge'] = df['to_edge'], df['from_edge']
# build a undirected graph
pd.concat([df,df_1], ignore_index=True).to_csv(f'../instances/{folder}/edge.csv', index=None)

In [15]:
folder = 'caltech'
rep = 1 # For illustration, generate one repetition. To fully replicate the result in the paper, change this to 50.
for i in range(1,rep+1):
    sim_id = f'{folder}/sim_{i}'
    os.makedirs(f'../instances/{sim_id}', exist_ok=True)
    params = {'sim_id':sim_id, 'alpha':0.1, 'beta':3.5,
              'noise_dist': 'logistic', 'noise_dist_param':1,
              'cnt_iter': 100000, 't_warmup':1000, 'is_traj':False, 
              'is_network_given':True, 'network_dir':f'../instances/{folder}/edge.csv',
              'is_value_given':False, 'v_dist':'uniform_neg', 'v_dist_param': 4
             }
    
    G = network(params)
    sample = diffusion_simulation(G,params)
    sample.start_diffusion()
    sample.run_fixed_point()
    sample.output()

Instance generated with 770 nodes.


In [21]:
df = pd.read_csv(f'../instances/{sim_id}/results.csv')
print('MAPE:', np.mean(np.abs(df['sim']-df['fp'])/df['sim'])*100)

MAPE: 3.4243050026408763
